In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, exposure, feature, morphology, measure, filters
from skimage.color import rgb2gray
import scipy.stats as stats
import sklearn.feature_extraction
from sklearn.model_selection import train_test_split
import random
from sklearn import svm, tree
import pickle
from keras.models import Sequential, Model, load_model
from keras.layers import concatenate, core, Flatten, Conv2D, Dense, MaxPooling2D, ThresholdedReLU, Dropout, Conv2DTranspose, UpSampling2D
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def imshow(img, title="", preview = True):
    if preview:
        plt.figure(figsize = (5,5))
        plt.title(title)
        plt.imshow(img, cmap='gray')
        plt.show()
        
def load(id, preview = False):
    img = plt.imread(f"chase/Image_{id}.jpg")
    imshow(img, "Obraz oryginalny", preview)
    
    ref = plt.imread(f"chase/Image_{id}_1stHO.png")
    imshow(ref, "Obraz referencyjny", preview)
    
    return (img, ref)

def mask(image):
    (h, w) = image.shape
    center = (int(w/2), int(h/2))
    radius = min(center[0], center[1], w-center[0], h-center[1]) * 0.92

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    image[dist_from_center > radius] = 0
    
def get_picture_data(img, ref, box=5):
    patches = sklearn.feature_extraction.image.extract_patches_2d(img, (box,box))
    
    if box % 2 == 0:
        to_slice = box // 2
        main_squares = ref[to_slice : -to_slice + 1, to_slice : -to_slice + 1].flatten()
    else:
        to_slice = box//2
        main_squares = ref[to_slice : -to_slice, to_slice : -to_slice].flatten()

    return (patches, main_squares)

def analysis(img, ref, file_name=None, method=""):
    (rows, columns) = ref.shape
    output_img = []
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    for r in range(rows):
        output_img.append([])
        for c in range(columns):
            if ref[r][c] == 1:
                if img[r][c] == 1:
                    tp += 1
                    output_img[r].append([0,1.0,0]) #green
                else:
                    fn += 1
                    output_img[r].append([1.0,0,0]) #red
            else:
                if img[r][c] == 1:
                    fp += 1
                    output_img[r].append([1.0,1.0,0]) #yellow
                else:
                    tn += 1
                    output_img[r].append([0,0,0]) #black
    pop = tp + fn + fp + tn
    
    accuracy = (tp + tn) / pop
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    balanced_accuracy = (sensitivity + specificity) / 2
    f1 = (2 * tp) / (2 * tp + fp + fn)
    
    output_img = np.array([np.array(x) for x in output_img])
    imshow(output_img, "Porównanie wyniku z obrazem referencyjnym", True)
    
    print(f"True positive:\t{tp}\t(zielony)")
    print(f"True negative:\t{tn}\t(czarny)")
    print(f"False positive:\t{fp}\t(żółty)")
    print(f"False negative:\t{fn}\t(czerwony)")
    print(f"Accuracy (trafność):\t\t{accuracy}")
    print(f"Sensitivity (czułość):\t\t{sensitivity}")
    print(f"Specificity (swoistość):\t{specificity}")
    print(f"Balanced accuracy:\t\t{balanced_accuracy}")
    print(f"F1 score:\t\t\t{f1}")
    
    if (file_name):
        filepath_name = "./raport/dane/" + method + "/" + file_name
        
        plt.imsave(filepath_name + ".png", output_img)
        
        file = open(filepath_name + ".txt", "w")
        file.write("\\begin{figure}[!h]\n")
        file.write("\t\\centering\n")
        file.write("\t\\begin{minipage}{0.26\\linewidth}\n")
        file.write("\t\t\\includegraphics[width=\\linewidth]{./dane/" + method + "/" + file_name + ".png}\n")
        file.write("\t\t\\centering\n")
        file.write("\t\t\t\\small{" + method + "}\n")
        file.write("\t\\end{minipage}\n")
        file.write("\t\\hfill\n")
        file.write("\t\\begin{minipage}{0.55\\linewidth}\n")
        file.write("\t\t\\begin{tabular}{|>{\columncolor[gray]{0.9}}c|c|c|}\n")
        file.write("\t\t\t\\hline \\rowcolor{Gray}\n")
        file.write("\t\t\t\\diagbox{\\textbf{pred.}}{\\textbf{act.}} & \\textbf{positives} & \\textbf{negatives} \\\\ \\hline\n") 
        file.write("\t\t\t\\textbf{positives} & " + str(tp))
        file.write(f"& {fp} \\\\ \\hline\n")
        file.write("\t\t\t\\textbf{negatives} & " + str(fn))
        file.write(f"& {tn} \\\\ \\hline\n")
        file.write("\t\t\\end{tabular}\n")
        file.write("\t\t\\begin{tabular}{l r}\n")
        file.write("\t\t\tAccuracy (trafność): & %.4f \\\\\n" % accuracy)
        file.write("\t\t\tSensitivity (czułość): & %.4f \\\\\n" % sensitivity)
        file.write("\t\t\tSpecificity (swoistość): & %.4f \\\\\n" % specificity)
        file.write("\t\t\tBalanced accuracy: & %.4f \\\\\n" % balanced_accuracy)
        file.write("\t\t\tF1 score: & %.4f \\\\\n" % f1) 
        file.write("\t\t\\end{tabular}\n")
        file.write("\t\\end{minipage}\n")
        file.write("\\end{figure}\n")
        file.close()

# Metoda 1

In [ ]:
def method_1(img, preview = False):
    
    img = rgb2gray(img)
    
    img = exposure.adjust_gamma(img, 0.5)
    imshow(img, "Gamma", preview)
    
    img = exposure.rescale_intensity(img)
    imshow(img, "Ekspozycja", preview)

    filtered = filters.gaussian(img, 3)
    imshow(filtered, "Rozmycie", preview)
    
    filtered = filters.unsharp_mask(img, 10)
    imshow(filtered, "Wyostrzenie", preview)

    edges = filters.frangi(filtered)
    imshow(edges, "Krawędzie", preview)

    binary = edges > filters.threshold_triangle(edges, 512)
    imshow(binary, "Maska binarna", preview)
    
    f = morphology.binary_opening(binary)
    imshow(f, "Opening", preview)
    
    mask(f)
    imshow(f, "Usunięcie okręgu", preview)
    
    return f

### Wykonanie

In [ ]:
(img, ref) = load("01L", True)

print("Metoda 1")
m1 = method_1(img, False)
analysis(m1, ref, "01L")

# Klasyfikator

In [ ]:
def features(frag):
    return frag.flatten().flatten()

def before_retouch(img):
    img = exposure.adjust_gamma(img, 0.5)
    img = exposure.rescale_intensity(img)
    filtered = filters.gaussian(img, 3, multichannel=True)
    filtered = filters.unsharp_mask(img, 10)
    return filtered

In [ ]:
def check_if_only_zeros(listt):
    for value in listt.flatten().flatten().flatten():
        if value != 0:
            return False
    return True

def beforeRetouch(img):
    img = exposure.adjust_gamma(img, 0.5)
    img = exposure.rescale_intensity(img)
    filtered = filters.gaussian(img, 3, multichannel=True)
    filtered = filters.unsharp_mask(img, 10)
    return filtered

def getTrainingDataFromImage( id, box, remove0=True, data5050=True ):
    (img, ref) = load(id, False)
    img = beforeRetouch(img)
    (patches, refs) = get_picture_data(img, ref, box)
    
    print("All patches, refs: ", len(patches), len(refs))
    
    if (remove0):
        indexesToRemove = []
        for i in range(len(patches)):
            if (check_if_only_zeros(patches[i]) ):
                indexesToRemove.append(i)

        indexesToRemove.pop(0)
        print("To remove (0,0,0): ", len(indexesToRemove))
        patches = np.delete(patches, indexesToRemove, axis=0)
        refs = np.delete(refs, indexesToRemove, axis=0)
        print("After removing: ", len(patches))

    if (data5050):
        refs_positive = refs[refs == True]
        patches_positive = patches[refs == True]

        refs_negative = refs[refs == False]
        patches_negative = patches[refs == False]
        print("Positive (patches, refs): ", len(patches_positive), len(refs_positive))
        print("Negative (patches, refs): ", len(patches_negative), len(refs_negative))

        sample_indexes = random.sample( range(len(refs_negative)), len(refs_positive))

        patches = np.append(patches_positive, patches_negative[sample_indexes], axis=0)
        refs = np.append(refs_positive, refs_negative[sample_indexes], axis = 0)
        print("Img data to sample from (patches, refs): ", len(patches), len(refs), "\n")
    
    feats = [features(f) for f in patches]
    return (feats, refs)

def getTrainingData( imgIdList, sampleCount, box=9, remove0=True, data5050=True ):
    allFeats = []
    allRefs = []
    for imgId in imgIdList:
        print("Loading", imgId)
        (feats, refs) = getTrainingDataFromImage(imgId, box, remove0, data5050)
        allFeats += list(feats)
        allRefs += list(refs)
        
    print("All img data to sample from: ",len(allFeats))
    
    return random.sample(list(zip(allFeats, allRefs)), sampleCount)

In [ ]:
def testfunction(estimator, id, box=9):
    (img, ref) = load(id, False)
    img = before_retouch(img)
    (patches, refs) = get_picture_data(img, ref, box)
    feats = [features(f) for f in patches]
    
    predicted = estimator.predict(feats)
    print('sum of predicted', sum(predicted))

    predicted_2d = np.reshape(predicted, (img.shape[0] - box + 1, img.shape[1] - box + 1))
    refs_2d = np.reshape(refs, (img.shape[0] - box + 1, img.shape[1] - box + 1))
    
    # predicted_2d = morphology.dilation(predicted_2d)

    return (predicted_2d, refs_2d)

In [ ]:
def method_2(id):
    clf = pickle.load(open("clf", "rb"))
    return testfunction(clf, id, box=9)

### Trenowanie

In [ ]:
sample = getTrainingData(["01L", "01R"], 100000, 5, remove0=False, data5050=True)

clf = tree.DecisionTreeClassifier()
clf.fit([f for f, _ in sample], [r for _, r in sample])

# Zapis klasyfikatora do pliku
pickle.dump(clf, open("clf", "wb"))

### Wykonanie

In [ ]:
# Odczyt klasyfikatora z pliku
clf = pickle.load(open("clf", "rb"))

(predicted_2d, ref_2d) = method_2("05L")
plt.imshow(predicted_2d, cmap='gray')

analysis(p2d, ref_2d)

# Sieć neuronowa

In [ ]:
def patch_preprocess(patch):
    m = np.mean(patch, axis=(0,1))
    s = np.std(patch, axis=(0,1))
    s[s == 0] = 0.000000001
    standarized = (patch - m) / s
    min_value = np.min(standarized, axis=(0,1))
    max_value = np.max(standarized, axis=(0,1))
    div = (max_value - min_value)
    div[div == 0] = 0.000000001
    return (standarized - min_value) / div

In [ ]:
def get_train_test_data(ids, sampleCount, test_size=0.33, box=5):
    allPatches = []
    allRefs = []
    for imgId in ids:
        print("Loading", imgId)
        (img, ref) = load(imgId, False)
        #img = before_retouch(img)
        (patches, refs) = get_picture_data(img, ref, box)
        allPatches.append(patches)
        allRefs.append(refs)
        
    # Join data from all images
    allPatches = np.concatenate(allPatches, axis=0)
    allRefs = np.concatenate(allRefs, axis=0)
    print(allPatches.shape)
    print(allRefs.shape)
    
    # Delete data
    indexes = random.sample(range(allPatches.shape[0] - 1), k=allPatches.shape[0] - sampleCount)
    allPatches = np.delete(allPatches, indexes, axis=0)
    allRefs = np.delete(allRefs, indexes, axis=0)
    
    allPatches = np.stack([patch_preprocess(patch) for patch in allPatches], axis=0)
    
    print(allPatches.shape)
    print(allRefs.shape)
    
    return train_test_split(allPatches, allRefs, test_size=0.33)

In [ ]:
# Test model
def test_model(model, id, box, threshold=None):
    (img, ref) = load(id, False)
    (patches, refs) = get_picture_data(img, ref, box)
    patches = np.stack([patch_preprocess(patch) for patch in patches], axis=0)
    predicted = model.predict(patches)
    predicted_2d = np.reshape(predicted, (img.shape[0] - box + 1, img.shape[1] - box + 1))
    refs_2d = np.reshape(refs, (img.shape[0] - box + 1, img.shape[1] - box + 1))
    if threshold is not None:
        predicted_2d = predicted_2d > threshold
    return (predicted_2d, refs_2d)

In [ ]:
def method_3(id):
    model = load_model('model.keras')
    return test_model(model, id, box=19, threshold=0.3)  

### Trenowanie

In [ ]:
# Pobieranie danych
x_train, x_test, y_train, y_test = get_train_test_data(["01L", "01R", "02L", "02R", "03L", "03R"], 800000, box=13)

In [ ]:
# Model
model = Sequential()

model.add(Conv2D(32, (3,3), activation = 'relu', padding='same', input_shape=(19,19,3)))
model.add(Conv2D(32, (3,3), activation = 'relu', padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(48, (3,3), activation = 'relu', padding='same'))
model.add(Conv2D(48, (3,3), activation = 'relu', padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu', padding='same'))
model.add(Conv2D(64, (3,3), activation = 'relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))

model.add(Conv2D(96, (3,3), activation = 'relu', padding='same'))
model.add(Conv2D(96, (3,3), activation = 'relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))

model.add(Conv2DTranspose(64, (2,2), strides=(2,2), activation = 'relu', padding='same'))
model.add(Conv2D(64, (3,3), activation = 'relu', padding='same'))
model.add(Conv2D(64, (3,3), activation = 'relu', padding='same'))

model.add(Conv2DTranspose(48, (2,2), strides=(2,2), activation = 'relu', padding='same'))
model.add(Conv2D(48, (3,3), activation = 'relu', padding='same'))
model.add(Conv2D(48, (3,3), activation = 'relu', padding='same'))

model.add(Conv2DTranspose(32, (2,2), strides=(2,2), activation = 'relu', padding='same'))
model.add(Conv2D(32, (3,3), activation = 'relu', padding='same'))
model.add(Conv2D(32, (3,3), activation = 'relu', padding='same'))

model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=256)

In [ ]:
model.save('model.keras')

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss = {loss*100.0}%\nAccuracy = {accuracy*100}%")

### Wykonanie

In [ ]:
(predicted, reference) = test_model(model, "04L", box=13, threshold=0.3)
imshow(predicted)

In [ ]:
analysis(predicted, reference)

### Wczytywanie

In [ ]:
import time

start = time.time()
(predicted, reference) = method_3("05L")
end = time.time()

print(end - start)

In [ ]:
imshow(predicted)

In [ ]:
analysis(predicted, reference)

# Uruchamianie

In [ ]:
def go_button_clicked(btn):
    global filename, methods, do_analysis
    name = filename.value
    method = methods.value
    doing_analysis = do_analysis.value
    if method == 1:
        (img, ref) = load(name, True)
        m1 = method_1(img, False)
        imshow(m1, "Metoda 1")
        if doing_analysis:
            analysis(m1, ref)
    elif method == 2:
        _ = load(name, True)
        (predicted, ref) = method_2(name)
        imshow(predicted, "Metoda 2")
        if doing_analysis:
            analysis(predicted, ref)
    elif method == 3:
        _ = load(name, True)
        (predicted, ref) = method_3(name)
        imshow(predicted, "Metoda 3")
        if doing_analysis:
            analysis(predicted, ref)

filename = widgets.Text(
    value='04L',
    description='Nazwa pliku:',
    disabled=False)

methods = widgets.RadioButtons(
    options=[('Metoda 1', 1), ('Metoda 2 (Klasyfikator)', 2), ('Metoda 3 (Sieć neuronowa)', 3)],
    value=1,
    description='Metoda:',
    disabled=False
)

do_analysis = widgets.Checkbox(
    value=False,
    description='Analiza',
    disabled=False)

button = widgets.Button(
    description='Wykryj',
    disabled=False)

button.on_click(go_button_clicked)

display(filename, methods, do_analysis, button)